In [1]:
import pandas as pd
import numpy as np
from prepare_data import prepare
from load_data import load

In [2]:
train, test, train_dict = load()

100%|████████████████████████████████████████████| 8/8 [00:08<00:00,  1.05s/it]


In [3]:
all_data = prepare(pd.concat([train, test]).reset_index(drop=True), train_dict)

train = all_data.loc[:train.shape[0] -1,:]
test = all_data.loc[train.shape[0]:,:] 